In [2]:
import os
from pydub import AudioSegment
from pydub.utils import make_chunks

# Function to hide message in the last bit of audio samples
def hide_message_lsb(audio, binary_message):
    samples = audio.get_array_of_samples()
    new_samples = samples[:]
    for i in range(len(binary_message)):
        bit = binary_message[i]
        new_samples[i] = (samples[i] & ~1) | int(bit)
    return audio._spawn(new_samples)

# Function to extract message hidden in the last bit of audio samples
def extract_message_lsb(audio, message_length):
    samples = audio.get_array_of_samples()
    extracted_bits = [str(samples[i] & 1) for i in range(message_length)]
    return ''.join(extracted_bits)

# Function to convert binary message string to bytes
def binary_to_bytes(binary_message):
    byte_chunks = [binary_message[i:i + 8] for i in range(0, len(binary_message), 8)]
    return bytes([int(byte, 2) for byte in byte_chunks])

# Function to convert bytes to binary message string
def bytes_to_binary(byte_data):
    return ''.join(format(byte, '08b') for byte in byte_data)

# Function to write binary message to a .bin file
def write_binary_message_to_bin(binary_message, file_path):
    byte_data = binary_to_bytes(binary_message)
    with open(file_path, 'wb') as file:
        file.write(byte_data)

# Function to calculate maximum embedding capacity of the cover audio
def calculate_max_capacity(audio):
    return len(audio.get_array_of_samples())

# Paths for input files and directories
binary_message_path = r'C:\Users\Hendy Group\OneDrive\Desktop\final_graduation_project\audio\example22.bin'
cover_audio_dir = r'C:\Users\Hendy Group\OneDrive\Desktop\final_graduation_project\cover_audio'
output_dir = r'C:\Users\Hendy Group\OneDrive\Desktop\final_graduation_project\audio'

# Read binary data from .bin file
with open(binary_message_path, 'rb') as file:
    binary_data = file.read()
binary_message = bytes_to_binary(binary_data)

# Determine the number of cover audio files needed
cover_audio_files = sorted([os.path.join(cover_audio_dir, f) for f in os.listdir(cover_audio_dir) if f.endswith('.mp3')])
num_cover_audios = len(cover_audio_files)

# Load the first cover audio to get its dimensions
cover_audio = AudioSegment.from_file(cover_audio_files[0])
max_capacity = calculate_max_capacity(cover_audio)

# Split binary message into chunks
chunk_size = max_capacity
binary_message_chunks = [binary_message[i:i + chunk_size] for i in range(0, len(binary_message), chunk_size)]
num_full_chunks = len(binary_message_chunks)

# Embed each chunk into a separate cover audio
modified_audios = []
for j, chunk in enumerate(binary_message_chunks):
    # Determine the current cover audio index using modulo operation
    i = j % num_cover_audios
    
    # Load the current cover audio
    cover_audio = AudioSegment.from_file(cover_audio_files[i])
    
    # Embed the message chunk into the current cover audio
    modified_audio = hide_message_lsb(cover_audio, chunk)
    modified_audios.append(modified_audio)
    
    # Save the modified audio with a sequential name
    output_audio_path = os.path.join(output_dir, f'{j + 1}.mp3')
    modified_audio.export(output_audio_path, format="mp3")
    print(f"Modified audio {j + 1} saved to: {output_audio_path}")

# Extract hidden message from each modified audio
extracted_messages = []
for j, modified_audio in enumerate(modified_audios):
    # Determine the current cover audio index using modulo operation
    i = j % num_cover_audios
    
    chunk_size = len(binary_message_chunks[j])
    extracted_message = extract_message_lsb(modified_audio, chunk_size)
    extracted_messages.append(extracted_message)

# Combine all extracted chunks into the complete binary message
extracted_message_lsb = ''.join(extracted_messages)

# Write extracted binary message to a .bin file
extracted_message_path = os.path.join(output_dir, 'extracted_message_0.bin')
write_binary_message_to_bin(extracted_message_lsb, extracted_message_path)
print(f"Extracted binary message saved to: {extracted_message_path}")

FileNotFoundError: [WinError 2] The system cannot find the file specified